In [12]:
#!pip install autogluon
#!pip install pandas

In [13]:
import pandas as pd
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection._split import _BaseKFold
import numpy as np
import random
from sklearn.model_selection import train_test_split

In [14]:
seed_value = 42  # Replace with your desired seed value
random.seed(seed_value)
np.random.seed(seed_value)

In [15]:
x_train_b = pd.read_csv('cleaned_data_quarterly/B/x_train_b.csv')
y_train_b = pd.read_csv('cleaned_data_quarterly/B/train_b.csv')
x_test_b = pd.read_csv('cleaned_data_quarterly/B/x_test_b.csv')

x_train_b

,date_forecast,absolute_humidity_2m:gm3_Q0,absolute_humidity_2m:gm3_Q1,absolute_humidity_2m:gm3_Q2,absolute_humidity_2m:gm3_Q3,air_density_2m:kgm3_Q0,air_density_2m:kgm3_Q1,air_density_2m:kgm3_Q2,air_density_2m:kgm3_Q3,ceiling_height_agl:m_Q0,...,forecast_day,forecast_hour,calc_year,calc_month,calc_day,calc_hour,direct_rad_x_sun_elevation_Q0,direct_rad_x_sun_elevation_Q1,direct_rad_x_sun_elevation_Q2,direct_rad_x_sun_elevation_Q3
0,2019-01-01 00:00:00,5.6,5.6,5.5,5.4,1.239,1.239,1.240,1.241,1226.4,...,1,0,NaN,NaN,NaN,NaN,-0.0000,-0.0,-0.0,-0.0
1,2019-01-01 01:00:00,5.4,5.4,5.4,5.5,1.241,1.240,1.239,1.239,1157.6,...,1,1,NaN,NaN,NaN,NaN,-0.0000,-0.0,-0.0,-0.0
2,2019-01-01 02:00:00,5.5,5.4,5.4,5.3,1.238,1.238,1.239,1.239,1087.6,...,1,2,NaN,NaN,NaN,NaN,-0.0000,-0.0,-0.0,-0.0
3,2019-01-01 03:00:00,5.3,5.3,5.4,5.4,1.240,1.240,1.240,1.239,1016.4,...,1,3,NaN,NaN,NaN,NaN,-0.0000,-0.0,-0.0,-0.0
4,2019-01-01 04:00:00,5.5,5.6,5.7,5.9,1.239,1.238,1.237,1.236,1029.0,...,1,4,NaN,NaN,NaN,NaN,-0.0000,-0.0,-0.0,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26937,2023-04-30 19:00:00,4.6,4.6,4.5,4.5,1.275,1.276,1.277,1.278,1636.1,...,30,19,2023.0,4.0,29.0,7.0,0.7652,0.0,-0.0,-0.0
26938,2023-04-30 20:00:00,4.5,4.5,4.5,4.5,1.278,1.279,1.279,1.279,1747.8,...,30,20,2023.0,4.0,29.0,7.0,-0.0000,-0.0,-0.0,-0.0
26939,2023-04-30 21:00:00,4.5,4.5,4.5,4.5,1.279,1.279,1.279,1.279,1797.7,...,30,21,2023.0,4.0,29.0,7.0,-0.0000,-0.0,-0.0,-0.0
26940,2023-04-30 22:00:00,4.5,4.5,4.5,4.5,1.279,1.280,1.280,1.280,1534.3,...,30,22,2023.0,4.0,29.0,7.0,-0.0000,-0.0,-0.0,-0.0


In [16]:
x_train_b_combined = x_train_b.merge(y_train_b, left_on='date_forecast', right_on='time', how='left')
x_train_b_combined['observed'] = x_train_b_combined['calc_year'].isna().astype(int)

train_data_b = x_train_b_combined.drop(['time', 'calc_year', 'calc_month', 'calc_day', 'calc_hour','date_forecast'], axis = 1)
x_test_b['observed'] = x_test_b['calc_year'].isna().astype(int)
test_data_b = x_test_b.drop([ 'calc_year', 'calc_month', 'calc_day', 'calc_hour','date_forecast'], axis = 1)

In [17]:
test_data_b

,location,absolute_humidity_2m:gm3_Q0,absolute_humidity_2m:gm3_Q1,absolute_humidity_2m:gm3_Q2,absolute_humidity_2m:gm3_Q3,air_density_2m:kgm3_Q0,air_density_2m:kgm3_Q1,air_density_2m:kgm3_Q2,air_density_2m:kgm3_Q3,ceiling_height_agl:m_Q0,...,wind_speed_w_1000hPa:ms_Q3,forecast_year,forecast_month,forecast_day,forecast_hour,direct_rad_x_sun_elevation_Q0,direct_rad_x_sun_elevation_Q1,direct_rad_x_sun_elevation_Q2,direct_rad_x_sun_elevation_Q3,observed
0,B,4.3,4.3,4.3,4.3,1.283,1.283,1.283,1.283,912.3,...,-0.0,2023,5,1,0,-0.0000,-0.000000,-0.0000,-0.00000,0
1,B,4.3,4.3,4.2,4.2,1.283,1.283,1.283,1.283,NaN,...,-0.0,2023,5,1,1,-0.0000,-0.000000,-0.0000,-0.00000,0
2,B,4.2,4.2,4.1,4.1,1.283,1.283,1.283,1.282,1482.8,...,-0.0,2023,5,1,2,-0.0000,-0.000000,-0.0000,-0.00000,0
3,B,4.1,4.0,4.0,4.0,1.282,1.282,1.282,1.283,2237.7,...,-0.0,2023,5,1,3,-0.0000,0.000000,5.4626,20.74660,0
4,B,3.9,3.9,3.9,3.9,1.283,1.282,1.282,1.281,2323.2,...,-0.0,2023,5,1,4,56.0780,125.151596,243.3750,419.51752,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,B,8.3,8.3,8.4,8.4,1.197,1.198,1.198,1.199,3692.2,...,0.0,2023,7,3,19,160.1557,131.985000,100.5885,58.52950,0
716,B,8.4,8.5,8.6,8.6,1.200,1.200,1.201,1.202,3553.3,...,0.0,2023,7,3,20,31.5776,15.477500,6.7200,2.01120,0
717,B,8.7,8.8,8.8,8.9,1.203,1.203,1.204,1.205,3014.1,...,0.0,2023,7,3,21,0.0000,-0.000000,-0.0000,-0.00000,0
718,B,9.0,9.0,9.0,9.0,1.206,1.206,1.206,1.206,2318.0,...,0.0,2023,7,3,22,-0.0000,-0.000000,-0.0000,-0.00000,0


# Trying with validation set equal to half of the estimated data


In [18]:

train_data_b.drop(columns= ['index'], inplace= True)

In [19]:
test_data_b['snow_density:kgm3_Q0'] = np.nan
test_data_b['snow_density:kgm3_Q1'] = np.nan
test_data_b['snow_density:kgm3_Q2'] = np.nan
test_data_b['snow_density:kgm3_Q3'] = np.nan

In [20]:
observed_zero = train_data_b[train_data_b['observed'] == 0]
observed_zero['forecast_month'].unique()


array([10, 11, 12,  1,  2,  3,  4], dtype=int64)

In [21]:
from sklearn.model_selection import train_test_split

def split_dataset(train_data, date_column='forecast_month', observed_column='observed'):
    """
    Splits the dataset into a training set and a validation set.
    The validation set includes approximately 50% of the estimated data, evenly distributed across months.
    Additionally, it includes about half of the observed data for May, June, and July, if present.
    The training set includes all months, excluding the observed data that is included in the validation set.
    
    :param train_data: The original training dataset as a pandas DataFrame.
    :param date_column: The name of the column that contains the month information.
    :param observed_column: The name of the column that indicates if the data is observed.
    :return: A tuple (training_set, validation_set)
    """
    # Work with a copy to avoid modifying the original DataFrame
    train_data = train_data.copy()
    train_data.sort_values(by='date_forecast', inplace=True)
    
    # Separate observed and estimated data
    estimated_data = train_data[train_data[observed_column] == 0]
    observed_data = train_data[train_data[observed_column] != 0]
    
    # Split the estimated data into training and validation sets
    estimated_train, estimated_val = train_test_split(
        estimated_data, test_size=0.5, random_state=42, stratify=estimated_data[date_column]
    )
    
    # Check if there are any observed data for May, June, and July
    if not observed_data[observed_data[date_column].isin([5, 6, 7])].empty:
        print("hellooo")
        observed_may_june_july = observed_data[observed_data[date_column].isin([5, 6, 7])]
        observed_train_mjj, observed_val_mjj = train_test_split(
            observed_may_june_july, test_size=0.5, random_state=42, stratify=observed_may_june_july[date_column]
        )
    else:
        
        observed_train_mjj = pd.DataFrame()
        observed_val_mjj = pd.DataFrame()
    
    # Combine the estimated and observed May, June, July data for the validation set
    validation_set = pd.concat([estimated_val, observed_val_mjj])
    validation_set.sort_values(by='date_forecast', inplace=True)
    
    # The rest of the observed data (excluding May, June, July) will be added to the training set
    observed_rest = observed_data[~observed_data[date_column].isin([5, 6, 7])]
    
    # Combine all training parts for the final training set
    training_set = pd.concat([estimated_train, observed_train_mjj, observed_rest])
    training_set.sort_values(by='date_forecast', inplace=True)
    
    return training_set, validation_set

# Create a copy of the DataFrame to pass to the function
train_data_copy = train_data_b.copy()
train_data_b, val_data_b = split_dataset(train_data_copy, 'forecast_month', 'observed')

print(len(train_data_b))
print(len(val_data_b))



In [22]:
predictor_b= TabularPredictor(label ='pv_measurement',eval_metric= 'mean_absolute_error').fit(train_data = train_data_b,tuning_data = val_data_b, use_bag_holdout=True, verbosity = 2,presets='best_quality',excluded_model_types=['CatBoost'], time_limit= 1500)

No path specified. Models will be saved in: "AutogluonModels\ag-20231107_120516\"
Presets specified: ['best_quality']
c:\Users\holwe\miniconda3\lib\site-packages\autogluon\core\utils\utils.py:549: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context("mode.use_inf_as_na", True):  # treat None, NaN, INF, NINF as NA
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 1500s
AutoGluon will save models to "AutogluonModels\ag-20231107_120516\"
AutoGluon Version:  0.8.2
Python Version:     3.9.7
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   691.42 GB / 1022.87 GB (67.6%)
Train Data Rows:    25141
Train Data Columns: 189
Tuning Data Rows:    1801
Tuning Data Columns: 189
Label Column: pv_measurement
Preprocessing data ...
c:\Users\holwe\mi

KeyboardInterrupt: 

In [ ]:
predictor_b.leaderboard(silent=True)

2023-11-06 17:33:51,525	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2023-11-06 17:33:51,542	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2023-11-06 17:33:51,565	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2023-11-06 17:33:51,590	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2023-11-06 17:33:51,598	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UN

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-9.705640,13.485852,1184.908182,0.000999,0.326635,3,True,11
1,CatBoost_BAG_L2,-9.826296,11.138754,1014.517941,0.244997,92.699295,2,True,9
2,LightGBMXT_BAG_L2,-9.826607,11.242417,989.127786,0.348659,67.309140,2,True,7
3,RandomForestMSE_BAG_L2,-10.147436,12.891197,1024.573112,1.997439,102.754465,2,True,8
4,ExtraTreesMSE_BAG_L2,-10.188498,13.418689,935.649194,2.524931,13.830548,2,True,10
5,LightGBMXT_BAG_L1,-10.399268,4.942795,583.517199,4.942795,583.517199,1,True,3
6,WeightedEnsemble_L2,-10.399268,4.944798,583.893363,0.002002,0.376164,2,True,6
7,CatBoost_BAG_L1,-13.536378,0.467589,226.815565,0.467589,226.815565,1,True,5
8,RandomForestMSE_BAG_L1,-13.545152,2.980001,111.135299,2.980001,111.135299,1,True,4
9,KNeighborsDist_BAG_L1,-17.877293,1.261168,0.152527,1.261168,0.152527,1,True,2


2023-11-06 17:33:51,605	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2023-11-06 17:33:51,640	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.


In [ ]:
nan_counts = x_train_b_combined.isna().sum()
print(nan_counts)

date_forecast                    0
absolute_humidity_2m:gm3_Q0      0
absolute_humidity_2m:gm3_Q1      0
absolute_humidity_2m:gm3_Q2      0
absolute_humidity_2m:gm3_Q3      0
                                ..
direct_rad_x_sun_elevation_Q3    0
time                             0
index                            0
pv_measurement                   0
observed                         0
Length: 197, dtype: int64


In [ ]:
y_pred_b = predictor_b.predict(test_data_b)
y_pred_b=pd.DataFrame(y_pred_b,columns=['pv_measurement'])
y_pred_b#print the DataFrame

,pv_measurement
0,0.101997
1,0.130705
2,0.264159
3,5.059661
4,48.364548
...,...
715,42.852310
716,13.377611
717,1.212325
718,0.469207


In [ ]:
y_pred_b = predictor_b.predict(test_data_b)
y_pred_b=pd.DataFrame(y_pred_b,columns=['pv_measurement'])
y_pred_b#print the DataFrame

,pv_measurement
0,0.101997
1,0.130705
2,0.264159
3,5.059661
4,48.364548
...,...
715,42.852310
716,13.377611
717,1.212325
718,0.469207


In [ ]:
import os
output_dir = 'autogluon'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the dataframes into the folder
y_pred_b.to_csv(os.path.join(output_dir, 'y_pred_b.csv'))